In [1]:
import sys
from pathlib import Path

In [2]:
src_path=Path("../src").resolve()
sys.path.append(str(src_path))

In [3]:
from sqlmodel import Session, select
from api.events.timescale_models import EventModel
from api.db.session import engine
from pprint import pprint

In [ ]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(100)
    compiled_query=query.compile(compile_kwargs={"literal_binds" : True})
    print(compiled_query)
    print("\n")
    pprint(str(query))

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 100


('SELECT eventmodel.id, eventmodel.time, eventmodel.page, '
 'eventmodel.description, eventmodel.updated_at \n'
 'FROM eventmodel ORDER BY eventmodel.updated_at ASC\n'
 ' LIMIT :param_1')


In [34]:
from timescaledb.hyperfunctions import time_bucket
from sqlalchemy import func
with Session(engine) as session:
    bucket=time_bucket("1 day" , EventModel.page)
    query = ( 
        select(bucket , EventModel.page , func.count() )
        .group_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query=query.compile(compile_kwargs={"literal_binds" : True})
    # result=session.exec(query).fetchall()

    print(compiled_query)
    print("\n")
    pprint(str(query))

SELECT time_bucket('1 day'::interval, eventmodel.page) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel GROUP BY time_bucket('1 day'::interval, eventmodel.page), eventmodel.page


("SELECT time_bucket('1 day'::interval, eventmodel.page) AS time_bucket_1, "
 'eventmodel.page, count(*) AS count_1 \n'
 "FROM eventmodel GROUP BY time_bucket('1 day'::interval, eventmodel.page), "
 'eventmodel.page')
